# ГлавРосГосНефть

Предположим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Шаги для выбора локации обычно такие:
- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

Нам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль.

## 1. Загрузите и подготовьте данные.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

geo_df0 = pd.read_csv('geo_data_0.csv')
geo_df1 = pd.read_csv('geo_data_1.csv')
geo_df2 = pd.read_csv('geo_data_2.csv')

geo_df0.info()
geo_df0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [2]:
geo_df1.info()
geo_df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [3]:
geo_df2.info()
geo_df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [4]:
print(geo_df0['id'].value_counts().head(15))
print(geo_df1['id'].value_counts().head())
print(geo_df2['id'].value_counts().head())

Tdehs    2
fiKDv    2
TtcGQ    2
74z30    2
HZww2    2
A5aEY    2
bxg6G    2
AGS9W    2
bsk9y    2
QcMuo    2
p2y4n    1
BGktw    1
aSBS6    1
GidX7    1
KSzqY    1
Name: id, dtype: int64
wt4Uk    2
LHZR0    2
5ltQ6    2
bfPNe    2
JRIE3    1
Name: id, dtype: int64
xCHr8    2
KUPhW    2
VF7Jo    2
Vcm5J    2
w5Xei    1
Name: id, dtype: int64


Пропусков в данных не обнаружено. При обучении модели от колонки с id можно избавиться: так не нужно будет проводить кодирование, так как отсутствуют качественные переменные. Хотя масштабировать признаки все таки придется. Также было обнаружено, что в каждом датасете присутствуют одни и те же скважины по несколько раз, хотя и с разными характеристиками. Таких данных немного, потому не будем их удалять, так как на модель они особо повлиять не смогут.

## 2. Обучите и проверьте модель для каждого региона.

In [5]:
#Избавились от колонки с id
geo_df0 = geo_df0.drop('id',axis=1)
geo_df1 = geo_df1.drop('id',axis=1)
geo_df2 = geo_df2.drop('id',axis=1)

#Разделяем датасеты на обычные и целевые признаки
target0   = geo_df0['product']
target1   = geo_df1['product']
target2   = geo_df2['product']
features0 = geo_df0.drop('product',axis=1)
features1 = geo_df1.drop('product',axis=1)
features2 = geo_df2.drop('product',axis=1)

#Разобьем данные на обучающую и валидационную выборки
features_train0, features_valid0, target_train0, target_valid0 = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

#Займемся масштабированием
scaler          = StandardScaler()
scaler.fit(features_train0)
features_train0 = scaler.transform(features_train0)
features_valid0 = scaler.transform(features_valid0)
scaler.fit(features_train1)
features_train1 = scaler.transform(features_train1)
features_valid1 = scaler.transform(features_valid1)
scaler.fit(features_train2)
features_train2 = scaler.transform(features_train2)
features_valid2 = scaler.transform(features_valid2)

#Теперь обучим линейную регрессию
#Первый регион
model0 = LinearRegression()
model0.fit(features_train0,target_train0)
predictions0 = model0.predict(features_valid0)
mean_product0 = np.round(predictions0.mean(),3)
rmse0 = np.round(np.sqrt(mean_squared_error(target_valid0,predictions0)),3)
print('Для первого региона: средний запас  = ' + str(mean_product0) + ' тыс. баррелей, RMSE модели = ' + str(rmse0))
#Второй регион
model1 = LinearRegression()
model1.fit(features_train1,target_train1)
predictions1 = model1.predict(features_valid1)
mean_product1 = np.round(predictions1.mean(),3)
rmse1 = np.round(np.sqrt(mean_squared_error(target_valid1,predictions1)),3)
print('Для второго региона: средний запас  = ' + str(mean_product1) + ' тыс. баррелей, RMSE модели = ' + str(rmse1))
#Третий регион
model2 = LinearRegression()
model2.fit(features_train2,target_train2)
predictions2 = model2.predict(features_valid2)
mean_product2 = np.round(predictions2.mean(),3)
rmse2 = np.round(np.sqrt(mean_squared_error(target_valid2,predictions2)),3)
print('Для третьего региона: средний запас = ' + str(mean_product2) + ' тыс. баррелей, RMSE модели = ' + str(rmse2))

Для первого региона: средний запас  = 92.593 тыс. баррелей, RMSE модели = 37.579
Для второго региона: средний запас  = 68.729 тыс. баррелей, RMSE модели = 0.893
Для третьего региона: средний запас = 94.965 тыс. баррелей, RMSE модели = 40.03


На первый взгляд, кажется, что явными фаворитами в выборе региона являются первый и третий. Однако, взглянув на метрику модели, виднеется явное отличие между вторым и другими регионами. Во втором - ошибка почти нулевая, значит модель для нее практически все предсказывает без ошибок.

## 3. Подготовьтесь к расчёту прибыли.

In [6]:
budget = 10**10
price_per_product =450000
#Расчитаем сколько в среднем должны приносить 200 лучших скважин в регионе, чтобы
#добыча была безубыточной
print(str(np.round(budget / 450000 / 200,2)) + ' тыс баррелей.')

111.11 тыс баррелей.


Определили то, сколько в среднем должно быть тысяч баррелей в лучших 200 скважинах, чтобы дабыча была безубыточной. Также сохранили значение бюджета на район и цену за 1 тыс. баррелей для дальнейших расчетов прибыли.

## 4. Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [7]:
def revenue(target,predictions,count):
    predictions = predictions.sort_values(ascending=False)[:count]
    top_200_oilers = target.iloc[predictions.index]
    product = top_200_oilers.sum()
    income = product * price_per_product
    return income - budget

In [8]:
#Протестируем функцию
profit0 = revenue(target_valid0,pd.Series(predictions0),200)
profit1 = revenue(target_valid1,pd.Series(predictions1),200)
profit2 = revenue(target_valid2,pd.Series(predictions2),200)
print('Прибыль в первом регионе  = {:,.2f} рублей'.format(profit0))
print('Прибыль во втором регионе = {:,.2f} рублей'.format(profit1))
print('Прибыль в третьем регионе = {:,.2f} рублей'.format(profit2))

Прибыль в первом регионе  = 3,320,826,043.14 рублей
Прибыль во втором регионе = 2,415,086,696.68 рублей
Прибыль в третьем регионе = 2,710,349,963.60 рублей


В данной функции мы берем первые 200 наиболее прибыльных скважин по предсказаниям модели, суммируем объем сырья, добытый с этих скважин и умножаем на цену продукта. Далее из полученного числа вычитаем бюджет на регион и получаем прибыль.

Также протестировали функцию, найдя прибыль по 200 лучшим предсказаниям в каждом из районов.

## 5. Посчитайте риски и прибыль для каждого региона.

In [12]:
#Задаем случайность
state = np.random.RandomState(12345)

profits0 = []

for i in range(1000):
    target_valid0 = target_valid0.reset_index()['product']
    target0_subsample = target_valid0.sample(500, replace=True, random_state=state)
    predictions0_subsample = pd.Series(predictions0[target0_subsample.index])
    profits0.append(revenue(target0_subsample,predictions0_subsample,200))

profits0 = pd.Series(profits0)
mean_profit0 = profits0.mean()
print('Средняя выручка в первом районе = {:,.2f} рублей'.format(mean_profit0))
confidence_interval0 = st.t.interval(.95,len(profits0)-1,mean_profit0,profits0.sem())
print('Доверительный интервал для первого района (interval)  = ({:,.2f}, {:,.2f})'.format(confidence_interval0[0],confidence_interval0[1]))

#Доверительный интервал через Bootstrap
vals0 = []

for i in range(1000):
    subsample0 = profits0.sample(frac=1, replace=True, random_state=state)
    vals0.append(subsample0.mean())
    
vals0 = pd.Series(vals0)
lower = vals0.quantile(.025)
upper = vals0.quantile(.975)
print('Доверительный интервал для первого района (Bootstrap) = ({:,.2f}, {:,.2f})'.format(lower,upper))

#Расчет рисков
neg_profits0 = profits0.loc[profits0 < 0]
risk0 = len(neg_profits0) / len(profits0) * 100
print('Риск для первого района = {:.1f}%'.format(risk0))

Средняя выручка в первом районе = 396,164,984.80 рублей
Доверительный интервал для первого района (interval)  = (379,620,315.15, 412,709,654.46)
Доверительный интервал для первого района (Bootstrap) = (379,886,701.79, 412,393,808.54)
Риск для первого района = 6.9%


In [14]:
profits1 = []

for i in range(1000):
    target_valid1 = target_valid1.reset_index()['product']
    target1_subsample = target_valid1.sample(500, replace=True, random_state=state)
    predictions1_subsample = pd.Series(predictions1[target1_subsample.index])
    profits1.append(revenue(target1_subsample,predictions1_subsample,200))

profits1 = pd.Series(profits1)
mean_profit1 = profits1.mean()
print('Средняя выручка во втором районе = {:,.2f} рублей'.format(mean_profit1))
confidence_interval1 = st.t.interval(.95,len(profits1)-1,mean_profit1,profits1.sem())
print('Доверительный интервал для первого района (interval)  = ({:,.2f}, {:,.2f})'.format(confidence_interval1[0],confidence_interval1[1]))

neg_profits1 = profits1.loc[profits1 < 0]
risk1 = len(neg_profits1) / len(profits1) * 100
print('Риск для второго района = {:.1f}%'.format(risk1))

Средняя выручка во втором районе = 439,360,809.14 рублей
Доверительный интервал для первого района (interval)  = (426,389,238.29, 452,332,379.99)
Риск для второго района = 2.3%


In [15]:
profits2 = []

for i in range(1000):
    target_valid2 = target_valid2.reset_index()['product']
    target2_subsample = target_valid2.sample(500, replace=True, random_state=state)
    predictions2_subsample = pd.Series(predictions2[target2_subsample.index])
    profits2.append(revenue(target2_subsample,predictions2_subsample,200))
   
profits2 = pd.Series(profits2)
mean_profit2 = profits2.mean()
print('Средняя выручка в третьем районе = {:,.2f} рублей'.format(mean_profit2))
confidence_interval2 = st.t.interval(.95,len(profits2)-1,mean_profit2,profits2.sem())
print('Доверительный интервал для первого района (interval)  = ({:,.2f}, {:,.2f})'.format(confidence_interval2[0],confidence_interval2[1]))


neg_profits2 = profits2.loc[profits2 < 0]
risk2 = len(neg_profits2) / len(profits2) * 100
print('Риск для третьего района = {:.1f}%'.format(risk2))

Средняя выручка в третьем районе = 398,572,852.63 рублей
Доверительный интервал для первого района (interval)  = (381,816,016.54, 415,329,688.71)
Риск для третьего района = 7.0%


В данном разделе проекта для каждого района мы построили распределение прибыли техникой Bootstrap. Через каждое распределение нашли среднюю прибыль и определили доверительный интервал. Для первого района нашли доверительный интервал двумя способами: функция interval и с помощью Bootstrap. В завершении оценили риски, найдя процентное соотношение отрицательной прибыли.

## Вывод

Обучив три модели линейной регрессии и найдя средний запас скважины для каждого региона, На первый взгляд, явным фаворитом является первый район. За ним идет третий район и в конце - второй. Однако, также взглянув на значение RMSE каждой модели, видим явное преимущество второго района по точности. Здесь ошибка почти нулевая.

Проведя дальнейшие расчеты прибыли по каждому району определили, что средняя выручка во втором районе оказалась на 40 млн. рубл., при наименьших рисках. Следовательно для добычи следует выбирать второй район.